In [4]:
import os

train_src_dir = "./src"
os.makedirs(train_src_dir, exist_ok=True)

In [4]:
!pip install azure-ai
!pip install azure-ai-ml

ERROR: Could not find a version that satisfies the requirement azure-ai (from versions: none)
ERROR: No matching distribution found for azure-ai


In [ ]:
!wget https://azuremlexamples.blob.core.windows.net/datasets/credit_card/default%20of%20credit%20card%20clients.csv

In the next cell, enter your Subscription ID, Resource Group name and Workspace name. To find these values:

1.    In the upper right Azure Machine Learning studio toolbar, select your workspace name.
2.    Copy the value for workspace, resource group and subscription ID into the code.
3.    You'll need to copy one value, close the area and paste, then come back for the next one.

In [3]:
from azureml.core import Workspace
from azureml.core import Experiment
ws = Workspace.from_config()


In [4]:

run = [ r for r in Experiment(ws, 'train-credit-cards').get_runs()]

In [5]:
from azureml.core import Workspace, Dataset

subscription_id = '827dbc18-97d7-4a9d-91d1-a0d0d15c9603'
resource_group = 'blazejtest'
workspace_name = 'blazejamlworkshop'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='credit_cards')

## Good, old fashioned model training

In [61]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

file_name = 'default of credit card clients.csv'
TEST_TRAIN_RATIO = 0.2
N_ESTIMATORS = 100
LEARNING_RATE = 1e-4

credit_df = dataset.to_pandas_dataframe()
credit_df = credit_df.drop("ID", axis=1)

train_df, test_df = train_test_split(
    credit_df,
    test_size=TEST_TRAIN_RATIO,
)
###################
#</prepare the data>
###################

##################
#<train the model>
##################
# extracting the label column
y_train = train_df.pop("default payment next month")

# convert the dataframe values to array
X_train = train_df.values

# extracting the label column
y_test = test_df.pop("default payment next month")

# convert the dataframe values to array
X_test = test_df.values

print(f"Training with data of shape {X_train.shape}")

clf = GradientBoostingClassifier(
    n_estimators=N_ESTIMATORS, learning_rate=LEARNING_RATE
)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

Training with data of shape (24000, 23)


In [11]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      1.00      0.87      4645
           1       0.00      0.00      0.00      1355

    accuracy                           0.77      6000
   macro avg       0.39      0.50      0.44      6000
weighted avg       0.60      0.77      0.68      6000



## 

In [30]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import mlflow
def train_model(TEST_TRAIN_RATIO, N_ESTIMATORS, LEARNING_RATE):
    mlflow.sklearn.autolog()
    credit_df = dataset.to_pandas_dataframe()

    train_df, test_df = train_test_split(
        credit_df,
        test_size=TEST_TRAIN_RATIO,
    )
    ###################
    #</prepare the data>
    ###################

    ##################
    #<train the model>
    ##################
    # extracting the label column
    y_train = train_df.pop("default payment next month")

    # convert the dataframe values to array
    X_train = train_df.values

    # extracting the label column
    y_test = test_df.pop("default payment next month")

    # convert the dataframe values to array
    X_test = test_df.values

    print(f"Training with data of shape {X_train.shape}")

    clf = GradientBoostingClassifier(
        n_estimators=N_ESTIMATORS, learning_rate=LEARNING_RATE
    )
    clf.fit(X_train, y_train)
    mlflow.sklearn.eval_and_log_metrics(clf, X_test, y_test, prefix="val_")

In [31]:
import mlflow

mlflow.set_tracking_uri(workspace.get_mlflow_tracking_uri())
experiment_name = "experiment"
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='', experiment_id='e050f288-2dba-4620-9d8f-6ec4d872aa86', lifecycle_stage='active', name='experiment', tags={}>

# To copy

In [121]:
import os
os.makedirs("./scripts", exist_ok=True)

In [141]:
%%writefile scripts/train.py 
## %%writefile creates a .py file from cell. This file will be written to scripts/train.py

import argparse # needed to parse arguments
from azureml.core import Run, Workspace # needed to fetch experiment context inside container
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
import argparse
import mlflow

def train_model_with_hyperparameters(TEST_TRAIN_RATIO, N_ESTIMATORS, LEARNING_RATE):
    mlflow.autolog() # start autologging
    
    run = Run.get_context() # get information about azure ml run
    dataset = run.input_datasets['credit_cards_ds']    
    with mlflow.start_run() as run:
        credit_df = dataset.to_pandas_dataframe()
        credit_df = credit_df.drop("ID", axis=1)
        
        ###################
        #</prepare the data>
        ###################
        train_df, test_df = train_test_split(
            credit_df,
            test_size=TEST_TRAIN_RATIO,
        )


        ##################
        #<train the model>
        ##################
        # extracting the label column
        y_train = train_df.pop("default payment next month")

        # convert the dataframe values to array
        X_train = train_df.values

        # extracting the label column
        y_test = test_df.pop("default payment next month")

        # convert the dataframe values to array
        X_test = test_df.values

        print(f"Training with data of shape {X_train.shape}")

        clf = GradientBoostingClassifier(
            n_estimators=N_ESTIMATORS, learning_rate=LEARNING_RATE
        )
        clf.fit(X_train, y_train)
        mlflow.sklearn.eval_and_log_metrics(clf, X_test, y_test, prefix="val_")

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--credit_cards_ds")
    parser.add_argument("--test_train_ratio", type=float)
    parser.add_argument("--n_estimators", type=int)
    parser.add_argument("--learning_rate", type=float)
    args = parser.parse_args()
    train_model_with_hyperparameters(
        args.test_train_ratio, 
        args.n_estimators, 
        args.learning_rate
    )


Overwriting scripts/train.py


In [142]:
%%writefile scripts/env.yml 
## %%writefile creates a file from cell. This file will be written to scripts/train.py

name: workshop-env
dependencies:
- python=3.8.5
- pip:
  - azureml-sdk==1.44.0
  - scikit-learn==0.22.1
  - scipy==1.5.3
  - numpy==1.21.6
  - azureml-mlflow
  - pandas==1.1.5


Overwriting scripts/env.yml


In [143]:
from azureml.core import ScriptRunConfig
from azureml.core import  Environment, Dataset


"""
ScriptRunConfig - encapsulates configuration of training script
Environment - encapsulates conda / pip env
"""


COMPUTE_TARGET = "testblazejamld" # replace with your own compute name
dataset = Dataset.get_by_name(ws, "credit_cards", version='latest')
env = Environment.from_conda_specification(
    name='workshop-env',
    file_path='./scripts/env.yml',
)

TEST_TRAIN_RATIO = 0.2
N_ESTIMATORS = 100
LEARNING_RATE = 1e-4

config = ScriptRunConfig(source_directory="./scripts", 
                      script='train.py', 
                      arguments =[
                        "--credit_cards_ds", dataset.as_named_input("credit_cards_ds"),
                        "--test_train_ratio", TEST_TRAIN_RATIO,
                        "--n_estimators", N_ESTIMATORS,
                        "--learning_rate", LEARNING_RATE,
                      ],
                      compute_target=COMPUTE_TARGET,
                      environment=env)


In [144]:
# it will take a bit longer first time since it has to create proper docker image
from azureml.core import Experiment
exp = Experiment(ws, 'containerized-experiment')
run = exp.submit(config=config)